Loading The data

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder .appName("TransportProject") .getOrCreate()


df = spark.read.parquet("/Users/lait-zet/Desktop/Syst-me-Pr-dictif-de-Transport-Urbain/data/Bronz/dataset.parquet")

df.show()



Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/08 11:04:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/01/08 11:04:50 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|cbd_congestion_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+
|       1| 2025-01-01 00:18:38|  2025-01-01 00:26:59|              1|          1.6|         1|                 N|         229|    

Checking Values Null

In [7]:
from pyspark.sql.functions import col, when, count,isnan

def check_missingValue(df):
    missing_counts = df.select([ count(when(col(c).isNull(), c)).alias(c) 
    for c in df.columns])
    return missing_counts


# missing_counts = check_missingValue(df)
# missing_counts.show()


Handle missing Data

In [8]:
from pyspark.sql.functions import mean, col, when, count,  unix_timestamp

def handel_missingValue(df):
    numeric_cols = []
    for name, dtype in df.dtypes:
        if dtype in ("int","float"):
            numeric_cols.append(name)
    
    for c in numeric_cols:
        mean_value = df.select(mean(c)).collect()[0][0]
        df = df.na.fill({c: mean_value})
    
    common_value = (
        df.groupBy("store_and_fwd_flag").count().orderBy(col("count").desc()).first()[0])
    df = df.na.fill({"store_and_fwd_flag": common_value})

    return df


# for c in data.columns:
#     null_count = data.select(count(when(col(c).isNull(), c))).collect()[0][0]
#     print(f"{c}: {null_count}")


Cheking Dublicate

In [9]:
def Handle_dublicate(df):
    total_rows = df.count()
    
    distinct_rows = df.distinct().count()
    
    dublicate_num = total_rows - distinct_rows

    return dublicate_num

dublicate = Handle_dublicate(df)


Duration Time

In [10]:
def Calcul_duration(df):
    df = df.withColumn(
    "duration_minute",
    (unix_timestamp("tpep_dropoff_datetime") - unix_timestamp("tpep_pickup_datetime")) / 60)
    return df

df = Calcul_duration(df)
df.show(10)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|cbd_congestion_fee|   duration_minute|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+------------------+
|       1| 2025-01-01 00:18:38|  2025-01-01 00:26:59|              1|    

Cheking The Condition -> Distance ≤ 0 ou > 200 miles | Durée ≤ 0 minutes | Passagers ≤ 0

In [11]:
def Handle_outliers(df):
    df_clean = df.filter((col("trip_distance") >= 0) & (col("trip_distance") <= 200) & (col("duration_minute") > 0) &(col("passenger_count") >= 0))
    return df_clean
df_clean = Handle_outliers(df)
# df_clean.count()

Features temporelles : pickuphour, dayof_week, month

In [12]:
from pyspark.sql.functions import hour,dayofweek,month
df_clean = df_clean.withColumn("pickuphour", hour(col("tpep_pickup_datetime")))
df_clean = df_clean.withColumn("dayof_week",dayofweek(col("tpep_pickup_datetime")))
df_clean = df_clean.withColumn("month", month(col("tpep_pickup_datetime")))

df_clean.show()


+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+------------------+----------+----------+-----+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|cbd_congestion_fee|   duration_minute|pickuphour|dayof_week|month|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+------------------+----------+-------

Save The Clean data

In [15]:
df_clean.coalesce(1).write.mode("overwrite").parquet("/Users/lait-zet/Desktop/Syst-me-Pr-dictif-de-Transport-Urbain/data/Silver")


Load the data again from Silver to check that saved correctly

In [16]:
new_data = spark.read.parquet("/Users/lait-zet/Desktop/Syst-me-Pr-dictif-de-Transport-Urbain/data/Silver/part-00000-06c3f70a-986d-4e1f-9358-9ec2d2161e5e-c000.snappy.parquet")
print(f"the data before saved {df_clean.count()}")
print(f"the data after saved {new_data.count()}")
print(df_clean.count() - new_data.count())

the data before saved 2933233
the data after saved 2933233
0
